<a href="https://colab.research.google.com/github/gzquse/GRADIENT_IMAGE/blob/main/Bell_CUDA_Q.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -v --no-cache-dir cuquantum-python

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
  Running command python setup.py egg_info
  [cuquantum-python] Trying to detect CUDA version from libraries: ['libnvrtc.so.12', 'libnvrtc.so.11.2', 'libnvrtc.so.11.1', 'libnvrtc.so.11.0']
  [cuquantum-python] Looking for library: libnvrtc.so.12
  [cuquantum-python] Detected version: 12020
  [cuquantum-python] Installing cuquantum-python-cu12...
  /usr/local/lib/python3.10/dist-packages/setuptools/__init__.py:84: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
  !!

          ********************************************************************************
          Requirements should be satisfied by a PEP 517 installer.
          If you are using pip, you can try `pip install --use-pep517`.
          ********************************************************************************

  !!
    dist.fetch_build_eggs(dist.setup_requires)
  /usr/local/lib/python3.10/dist-packages/s

In [4]:
!pip install -v --no-cache-dir qiskit-aer-gpu

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.8/18.8 MB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 139.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 895.1/895.1 kB 347.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 215.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.3/363.3 MB 271.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.3/130.3 MB 180.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 MB 155.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 329.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 206.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 264.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 288.4 MB/s eta 0:00:00
     ━━━━━━━━

In [7]:
import numpy as np
import cupy as cp

import cuquantum
from cuquantum import custatevec as cusv


nIndexBits = 3
nSvSize    = (1 << nIndexBits)
nMaxShots  = 5
nShots     = 5

bitStringLen  = 2;
bitOrdering   = np.asarray([0, 1], dtype=np.int32)

bitStrings = np.empty((nShots,), dtype=np.int64)
bitStrings_expected = np.asarray([0b00, 0b01, 0b10, 0b11, 0b11], dtype=np.int64)

h_sv          = np.asarray([0.0+0.0j, 0.0+0.1j, 0.1+0.1j, 0.1+0.2j,
                            0.2+0.2j, 0.3+0.3j, 0.3+0.4j, 0.4+0.5j], dtype=np.complex64)

d_sv = cp.asarray(h_sv)

# In real appliction, random numbers in range [0, 1) will be used.
randnums      = np.asarray([0.1, 0.8, 0.4, 0.6, 0.2], dtype=np.float64)

########################################################################

# cuStateVec handle initialization
handle = cusv.create()

# create sampler and check the size of external workspace
sampler, extraWorkspaceSizeInBytes = cusv.sampler_create(
    handle, d_sv.data.ptr, cuquantum.cudaDataType.CUDA_C_32F, nIndexBits, nMaxShots)

# allocate external workspace
extraWorkspace = cp.cuda.alloc(extraWorkspaceSizeInBytes)

# sample preprocess
cusv.sampler_preprocess(
    handle, sampler, extraWorkspace.ptr, extraWorkspaceSizeInBytes)

# sample bit strings
cusv.sampler_sample(
    handle, sampler, bitStrings.ctypes.data, bitOrdering.ctypes.data, bitStringLen,
    randnums.ctypes.data, nShots, cusv.SamplerOutput.ASCENDING_ORDER)

# destroy sampler
cusv.sampler_destroy(sampler)

# destroy handle
cusv.destroy(handle)

if not np.allclose(bitStrings, bitStrings_expected):
    raise ValueError("results mismatch")
print("test passed")

test passed


In [5]:
# Initial dictionary
initial_dict = {
    "0000": 675, "1001": 172, "0101": 498, "1101": 1508, "1110": 2677,
    "1100": 1066, "0110": 2949, "1000": 1302, "1010": 3213, "0010": 3508,
    "1011": 3210, "0011": 3515, "0111": 2949, "1111": 2705, "0001": 408, "0100": 1645
}

# Raw string
raw_string = "{ 0101:3359 1111:2680 0001:1285 0111:2641 1011:1610 0100:3420 0000:715 0011:1088 1000:461 1101:3237 1100:3389 0110:2915 0010:1671 1010:472 1001:166 1110:2891 }"

# Convert raw string to dictionary
raw_string = raw_string.strip("{ }")
raw_list = raw_string.split()
print(raw_list)
mapped_dict = {}
for item in raw_list:
    key, value = item.split(":")
    mapped_dict[key] = int(value)

# Function to reverse the keys
def reverse_key(key):
    return key[::-1]

# Create a new dictionary with reversed keys
reversed_initial_dict = {reverse_key(k): v for k, v in initial_dict.items()}

# Mapping function
def map_keys(initial, mapped):
    new_dict = {}
    for key in initial:
        if key in mapped:
            new_dict[key] = mapped[key]
    return new_dict

# Perform the mapping
mapped_reversed_initial_dict = map_keys(reversed_initial_dict, mapped_dict)

# Output the result
print("Mapped Dictionary:", mapped_reversed_initial_dict)


['0101:3359', '1111:2680', '0001:1285', '0111:2641', '1011:1610', '0100:3420', '0000:715', '0011:1088', '1000:461', '1101:3237', '1100:3389', '0110:2915', '0010:1671', '1010:472', '1001:166', '1110:2891']
Mapped Dictionary: {'0000': 715, '1001': 166, '1010': 472, '1011': 1610, '0111': 2641, '0011': 1088, '0110': 2915, '0001': 1285, '0101': 3359, '0100': 3420, '1101': 3237, '1100': 3389, '1110': 2891, '1111': 2680, '1000': 461, '0010': 1671}


In [6]:
import traceback

def main():
    # Initial list of dictionaries
    dict_list = [
        {"0000": 675, "1001": 172, "0101": 498, "1101": 1508, "1110": 2677,
         "1100": 1066, "0110": 2949, "1000": 1302, "1010": 3213, "0010": 3508,
         "1011": 3210, "0011": 3515, "0111": 2949, "1111": 2705, "0001": 408, "0100": 1645},
        # Add more dictionaries as needed
    ]

    # Raw string
    raw_string = "0101:3258 1111:2719 0001:1245 0111:2691 1011:1566 0100:3422 0000:727 0011:1060 1000:451 1101:3252 1100:3451 0110:2783 0010:1690 1010:458 1001:166 1110:3061 }"

    # Function to convert raw string to dictionary
    def string_to_dict(raw_string):
        raw_string = raw_string.strip("{ }")  # Correctly strip both curly brackets
        raw_list = raw_string.split()
        result = {}
        for item in raw_list:
            try:
                key, value = item.split(":")
                result[key] = int(value)
            except ValueError as e:
                print(f"Error processing item: {item}")
                traceback.print_exc()
                raise
        return result

    # Convert the raw string to a dictionary
    try:
        mapped_dict = string_to_dict(raw_string)
        print("Mapped Dictionary:", mapped_dict)
    except Exception as e:
        print("An error occurred while converting the raw string to a dictionary:")
        traceback.print_exc()

    # Function to reverse the keys
    def reverse_key(key):
        return key[::-1]

    # Function to process each dictionary
    def process_dict(d):
        try:
            # Reverse the keys in the initial dictionary
            reversed_dict = {reverse_key(k): v for k, v in d.items()}
            # Map the reversed keys to the values in the mapped_dict
            mapped_reversed_dict = {k: mapped_dict[k] for k in reversed_dict if k in mapped_dict}
            return mapped_reversed_dict
        except Exception as e:
            print(f"An error occurred while processing the dictionary: {d}")
            traceback.print_exc()
            raise

    # Apply the function to each dictionary in the list
    try:
        processed_dict_list = [process_dict(d) for d in dict_list]
    except Exception as e:
        print("An error occurred while processing the list of dictionaries:")
        traceback.print_exc()

    # Output the result
    try:
        for i, processed_dict in enumerate(processed_dict_list):
            print(f"Processed Dictionary {i+1}:", processed_dict)
    except Exception as e:
        print("An error occurred while printing the processed dictionaries:")
        traceback.print_exc()

if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        print("An unexpected error occurred:")
        traceback.print_exc()


Mapped Dictionary: {'0101': 3258, '1111': 2719, '0001': 1245, '0111': 2691, '1011': 1566, '0100': 3422, '0000': 727, '0011': 1060, '1000': 451, '1101': 3252, '1100': 3451, '0110': 2783, '0010': 1690, '1010': 458, '1001': 166, '1110': 3061}
Processed Dictionary 1: {'0000': 727, '1001': 166, '1010': 458, '1011': 1566, '0111': 2691, '0011': 1060, '0110': 2783, '0001': 1245, '0101': 3258, '0100': 3422, '1101': 3252, '1100': 3451, '1110': 3061, '1111': 2719, '1000': 451, '0010': 1690}


In [12]:
# Raw string
raw_string = "{ 0101:3207 1111:2626 0001:1216 0111:2656 1011:1532 0100:3429 0000:681 0011:1129 1000:448 1101:3280 1100:3520 0110:2921 0010:1715 1010:504 1001:174 1110:2962 }"

# Function to convert raw string to dictionary
raw_string = ''.join(filter(lambda x: x.isdigit() or x == ':' or x.isspace(), raw_string))
raw_string

' 0101:3207 1111:2626 0001:1216 0111:2656 1011:1532 0100:3429 0000:681 0011:1129 1000:448 1101:3280 1100:3520 0110:2921 0010:1715 1010:504 1001:174 1110:2962 '

In [13]:
import logging

# Configure logging
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("app.log"),
        logging.StreamHandler()
    ]
)

logger = logging.getLogger(__name__)

def some_function():
    try:
        logger.debug("This is a debug message")
        logger.info("This is an info message")
        logger.warning("This is a warning message")
        result = 10 / 0  # This will raise a ZeroDivisionError
    except ZeroDivisionError as e:
        logger.error("An error occurred: %s", e, exc_info=True)

if __name__ == "__main__":
    logger.info("Starting the application")
    some_function()
    logger.info("Finished the application")


2024-06-18 22:22:11,900 - __main__ - INFO - Starting the application
2024-06-18 22:22:11,901 - __main__ - DEBUG - This is a debug message
2024-06-18 22:22:11,902 - __main__ - INFO - This is an info message
2024-06-18 22:22:11,902 - __main__ - WARNING - This is a warning message
2024-06-18 22:22:11,902 - __main__ - ERROR - An error occurred: division by zero
Traceback (most recent call last):
  File "/tmp/ipykernel_1375791/2667594509.py", line 20, in some_function
    result = 10 / 0  # This will raise a ZeroDivisionError
             ~~~^~~
ZeroDivisionError: division by zero
2024-06-18 22:22:11,903 - __main__ - INFO - Finished the application


In [17]:
!pip install coloredlogs

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 650.4 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 1.6 MB/s eta 0:00:00eta 0:00:01


In [2]:
import logging
import traceback
import coloredlogs

# Configure logging
logging.basicConfig(
    filename='example.log',
    level=logging.DEBUG,
    format='%(levelname)-10s %(asctime)s %(module)s:%(lineno)d %(funcName)s %(message)s'
)

# Set up colored logs for console output
coloredlogs.install(
    level='DEBUG',
    fmt='%(levelname)-10s %(asctime)s %(module)s:%(lineno)d %(funcName)s %(message)s'
)

logger = logging.getLogger('my_logger')

def log_exception(e):
    # Capture and log the full traceback
    tb_str = ''.join(traceback.format_exception(type(e), e, e.__traceback__))
    logger.error(f"An error occurred: {e}\nTraceback:\n{tb_str}")

def some_function():
    try:
        logger.debug("This is a debug message")
        logger.info("This is an info message")
        logger.warning("This is a warning message")
        result = 10 / 0  # This will raise a ZeroDivisionError
    except ZeroDivisionError as e:
        log_exception(e)

def another_function():
    try:
        logger.debug("Attempting to read a non-existent file")
        with open('non_existent_file.txt', 'r') as f:
            data = f.read()
    except FileNotFoundError as e:
        log_exception(e)

def internal_error():
    try:
        logger.debug("Generating an internal error")
        raise ValueError("This is an internal error for testing purposes")
    except ValueError as e:
        log_exception(e)

if __name__ == "__main__":
    logger.info("Starting the application")
    some_function()
    another_function()
    internal_error()
    logger.info("Finished the application")


INFO       2024-06-19 07:46:37 1131554974:50 <module> Starting the application
DEBUG      2024-06-19 07:46:37 1131554974:27 some_function This is a debug message
INFO       2024-06-19 07:46:37 1131554974:28 some_function This is an info message
WARNING    2024-06-19 07:46:37 1131554974:29 some_function This is a warning message
ERROR      2024-06-19 07:46:37 1131554974:23 log_exception An error occurred: division by zero
Traceback:
Traceback (most recent call last):
  File "/tmp/ipykernel_2077756/1131554974.py", line 30, in some_function
    result = 10 / 0  # This will raise a ZeroDivisionError
             ~~~^~~
ZeroDivisionError: division by zero

DEBUG      2024-06-19 07:46:37 1131554974:36 another_function Attempting to read a non-existent file
ERROR      2024-06-19 07:46:37 1131554974:23 log_exception An error occurred: [Errno 2] No such file or directory: 'non_existent_file.txt'
Traceback:
Traceback (most recent call last):
  File "/tmp/ipykernel_2077756/1131554974.py", line 37